In [1]:
import pandas as pd
from sqlite3 import connect

**1. Создайте соединение с базой данных с помощью библиотеки sqlite3.**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
con = connect('/content/drive/MyDrive/Школа 21/Проверка/project11/data/checking-logs.sqlite')

**2. Используя только один запрос для каждой из групп, создайте два датафрейма: test_results и control_results со столбцами time и avg_diff и только двумя строками.**
*   times должно иметь значения before и after.
*   avg_diff содержит среднее значение дельты для всех пользователей за период времени до первого посещения ленты новостей каждым из них и после этого.
*   Учитываются только те пользователи, для которых имеются наблюдения и before, и after

In [4]:
row_sql = '''
SELECT time, avg(diff) from
	(SELECT uid, time, avg(diff) AS diff from
		(SELECT 
	        uid,
	        (strftime('%s', first_commit_ts) - deadlines)/60/60 diff,
	        case first_commit_ts > first_view_ts 
	        WHEN 1 THEN 'after' ELSE 'before'
	        END time
	    FROM test
	    JOIN deadlines
	    ON labname = labs
	    WHERE
	        labs <> 'project1' 
	        AND uid in 
	        (SELECT uid FROM test  
	        	  WHERE first_commit_ts > first_view_ts
	        		AND labname != 'project1'
			intersect	
			SELECT uid FROM test 
	        	WHERE first_commit_ts <= first_view_ts
	        		AND labname != 'project1')
	    )		
    GROUP BY 
    	uid, time
	)
GROUP BY  
	time
'''

In [5]:
test_results = pd.read_sql(row_sql, con=con)

In [6]:
test_results

,time,avg(diff)
0,after,-99.523810
1,before,-66.047619


In [7]:
row_sql = '''
SELECT time, AVG(diff) from
	(SELECT uid, time, AVG(diff) AS diff from
		(SELECT 
	        uid,
	        (strftime('%s', first_commit_ts) - deadlines)/60/60 diff,
	        case first_commit_ts > first_view_ts 
	        WHEN 1 then 'after' ELSE 'before'
	        END time
	    FROM control
	    JOIN deadlines
	    ON labname = labs
	    WHERE
	        labs <> 'project1' 
	        AND uid in
			(SELECT uid FROM control  
	        	WHERE first_commit_ts > first_view_ts
	        		AND labname != 'project1'
			intersect
			SELECT uid FROM control 
	        	WHERE first_commit_ts <= first_view_ts
	        	AND labname != 'project1')
	    )		
    GROUP BY 
    	uid, time
	)
GROUP BY
	time
'''

In [8]:
control_results = pd.read_sql(row_sql, con=con)

In [9]:
control_results

,time,AVG(diff)
0,after,-99.322222
1,before,-98.033333


In [10]:
con.close()

**5. Дайте ответ: оказалось ли предположение верным и влияет ли наличие страницы с новостной лентой на поведение учащихся?**

**Да, предположение было верным - наличие страницы с новостной лентой влияет на поведение учащихся.**